In [1]:
!pip install datasets transformers accelerate tokenizers

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer,Trainer, TrainingArguments

In [2]:
from datasets import load_dataset

In [3]:
datasets=load_dataset("text",data_files='/content/football.txt')

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15
    })
})

In [7]:
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token=tokenizer.eos_token

In [9]:
model=GPT2LMHeadModel.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
def tokenize_function(examples):
  tokenizer_inputs=tokenizer(examples["text"],truncation=True,padding="max_length",max_length=128)
  tokenizer_inputs["labels"]=tokenizer_inputs["input_ids"].copy()
  return tokenizer_inputs

In [11]:
tokenized_dataset=datasets.map(tokenize_function,batched=True,num_proc=4,remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/15 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15
    })
})

In [13]:
from peft import get_peft_model,LoraConfig,TaskType

In [14]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [15]:
peft_model=get_peft_model(model,lora_config)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [16]:
peft_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


In [20]:
training_args=TrainingArguments(
    output_dir="./Lora_football",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2
)

In [21]:
trainer=Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

In [22]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:



KeyboardInterrupt: 

In [23]:
model.save_pretrained("./Lora_football")

In [ ]:
input_prompt="When was world cup"

from transformers import pipeline

generator=pipeline("text-generation",model="./Lora_football",tokenizer=tokenizer)


result=generator(input_prompt,max_length=100,do_sample=True)
print(result[0]["generated_text"])

Some weights of the model checkpoint at ./Lora_football were not used when initializing GPT2LMHeadModel: ['transformer.h.0.attn.c_attn.base_layer.bias', 'transformer.h.0.attn.c_attn.base_layer.weight', 'transformer.h.0.attn.c_attn.lora_A.default.weight', 'transformer.h.0.attn.c_attn.lora_B.default.weight', 'transformer.h.1.attn.c_attn.base_layer.bias', 'transformer.h.1.attn.c_attn.base_layer.weight', 'transformer.h.1.attn.c_attn.lora_A.default.weight', 'transformer.h.1.attn.c_attn.lora_B.default.weight', 'transformer.h.10.attn.c_attn.base_layer.bias', 'transformer.h.10.attn.c_attn.base_layer.weight', 'transformer.h.10.attn.c_attn.lora_A.default.weight', 'transformer.h.10.attn.c_attn.lora_B.default.weight', 'transformer.h.11.attn.c_attn.base_layer.bias', 'transformer.h.11.attn.c_attn.base_layer.weight', 'transformer.h.11.attn.c_attn.lora_A.default.weight', 'transformer.h.11.attn.c_attn.lora_B.default.weight', 'transformer.h.2.attn.c_attn.base_layer.bias', 'transformer.h.2.attn.c_attn.ba